In [11]:
#存入URL
import requests
from bs4 import BeautifulSoup as bs
import string
from math import ceil
headers = {'cookie':'TRID_G=8711a259-e871-4895-bfc8-ef664f999722; SEID_G=bf7c90c9-369e-4672-9cb5-8c62da63d998; ASP.NET_SessionId=pos1kcp2zp0elm1vbhb34ujn; _ga=GA1.3.988170899.1438438756; WMX_Channel=,3,; __ltmwga=utmcsr=(direct)|utmcmd=(none); _gat_UA-22823074-5=1; CSCWORM_UNID=1440421905627627; _dc_gtm_UA-22823074-5=1; _dc_gtm_UA-34471860-1=1; _gat_UA-34471860-1=1; objID=; sto-id-%3Fv30%3FGroup-buy.housefun.com.tw%3A80=EFABBOKM; _ga=GA1.4.988170899.1438438756; __asc=5fa575a314f5fd6171a385af309; __auc=3ce783c214ee9a18edcba1d0ab5'}
count = requests.get('http://buy.housefun.com.tw/ashx/buy/new/GetBuyListTabCaseCount.ashx?callback=angular.callbacks._0&RequestPackage=%7B%22Method%22:%22INQUIRE%22,%22Data%22:%7B%22SID%22:%2282944189%22%7D%7D',headers=headers)
soup = bs(count.text)
cases = ''.join(soup.text.split('\"')[25].split(','))
page = int(ceil(float(cases)/10))

f = open('housefunXinbei_URL.txt','w')
page_html = "http://buy.housefun.com.tw/%E8%B2%B7%E5%B1%8B/%E6%96%B0%E5%8C%97%E5%B8%82?hd_CityID=0001&hd_Purpose=K&hd_Sequence=SEQUP&hd_SearchGroup=Group01&hd_PM={0}&hd_Tab=1&hd_SID=82963236"
for pg in range(1,page+1):
    res = requests.get(page_html.format(pg),headers=headers)
    soup = bs(res.text)
    for i in range(0,10):
        f.write("http://buy.housefun.com.tw" + soup.select('.m-list-obj')[i].select('a')[0]['href']+'\n')
f.close()

KeyboardInterrupt: 

In [21]:
#存入各物件html
import requests
headers = {'cookie':'TRID_G=8711a259-e871-4895-bfc8-ef664f999722; SEID_G=bf7c90c9-369e-4672-9cb5-8c62da63d998; ASP.NET_SessionId=pos1kcp2zp0elm1vbhb34ujn; _ga=GA1.3.988170899.1438438756; WMX_Channel=,3,; __ltmwga=utmcsr=(direct)|utmcmd=(none); _gat_UA-22823074-5=1; CSCWORM_UNID=1440421905627627; _dc_gtm_UA-22823074-5=1; _dc_gtm_UA-34471860-1=1; _gat_UA-34471860-1=1; objID=; sto-id-%3Fv30%3FGroup-buy.housefun.com.tw%3A80=EFABBOKM; _ga=GA1.4.988170899.1438438756; __asc=5fa575a314f5fd6171a385af309; __auc=3ce783c214ee9a18edcba1d0ab5'}
dir = 'housefunXinbei'
f = open("housefunXinbei_URL.txt","r")
for line in f:
    file_name = line.strip().split('/')[5].split('.')[0]
    input_file = open(dir+'/'+file_name+'.txt','w')
    
    res = requests.get(line.strip(),headers=headers)
    response = res.text.encode('utf8')
    input_file.write(response)
f.close()

In [59]:
import requests
from bs4 import BeautifulSoup as bs
import re
import glob
import os
import sys
from math import ceil
reload(sys)
sys.setdefaultencoding('utf8')

pk = ''
road = ''
latitude = ''
longitude = ''
price = ''
price_per =''
Housing_pattern_bedroom = ''
Housing_pattern_hall = ''
Housing_pattern_toilet = ''
age = ''
floor = ''
building_height = ''
building_type = ''
elevator = ''
orientation = ''
area_total = ''
main_building = ''
establishes_with_whole = ''
parking_space = ''
community = ''
building_structure = ''

ff = open('housefunxinbei_detail.txt','w')
dir_path = "housefunXinbei"
for file_name in glob.glob(os.path.join(os.getcwd(), dir_path, "*.txt")):
    f = open(file_name, "r")
    soup = bs(f.read())
    try:
        
        pk = soup.select('#query-hf-id')[0]['value']
        latitude = soup.select('#query-lat')[0]['value']
        longitude = soup.select('#query-lng')[0]['value']
        price = ''.join(soup.select('#total-amount')[0].text.split(','))
    
        string = ' '.join(soup.select('.info-sub')[0].text.split())
    
#格局
        Housing_pattern_bedroom = re.search('([\d]*)房',str(string.encode('utf8'))).group(1)
        if re.search('([\d]*)廳',str(string.encode('utf8'))):
            Housing_pattern_hall = re.search('([\d]*)廳',str(string.encode('utf8'))).group(1)
        else:Housing_pattern_hall = ''
        Housing_pattern_toilet = re.search('([\d\.]*)衛',str(string.encode('utf8'))).group(1)
#總坪數

        area_total = re.search('總坪數([\d\.]*) ',str(string.encode('utf8'))).group(1)

#每坪單價    
        if re.search('單價([\d\.]*)萬',str(string.encode('utf8'))):
            price_per = re.search('單價([\d\.]*)萬',str(string.encode('utf8'))).group(1)
        #elif  re.search('總坪數0',str(string.encode('utf8')))   
        else: price_per = str(round(float(price)/float(area_total),2))
#路名    
        road = soup.select('.extra-info')[0].select('li')[0].text.encode('utf8')
    
        extra = ''
        for ele in soup.select('.extra-info')[0].select('li'):
            extra = extra + ' ' + ' '.join(ele.text.split())
        
#樓層/樓高
        if re.search('樓層 ([\d]*)',str(extra.encode('utf8'))):
            floor = re.search('樓層 ([\d]*)',str(extra.encode('utf8'))).group(1)
        elif re.search('樓層 ([\S]*)',str(extra.encode('utf8'))):
            floor = re.search('樓層 ([\S]*)',str(extra.encode('utf8'))).group(1)
        else: floor = ''
        
        if re.search('/ ([\d]*)',str(extra.encode('utf8'))):
            building_height = re.search('/ ([\d]*)',str(extra.encode('utf8'))).group(1)
        else: building_height = ''
#屋齡
        if re.search('屋齡 ([\d]*)',str(extra.encode('utf8'))):
            age = re.search('屋齡 ([\d]*)',str(extra.encode('utf8'))).group(1)
        else: age = ''
#類型
        if re.search('公寓',str(extra.encode('utf8'))):
            building_type = unicode('公寓','utf8')
        elif re.search('大樓',str(extra.encode('utf8'))):
            building_type = unicode('大樓','utf8')
        elif re.search('透天厝',str(extra.encode('utf8'))):
            building_type = unicode('透天厝','utf8')
        else: building_type =''
#方位
        if re.search('朝向([\S]*) ',str(extra.encode('utf8'))): 
            orientation = unicode(re.search('朝向([\S]*) ',str(extra.encode('utf8'))).group(1),'utf8')
        else: orientation = ' '

        ff.write(pk+','+latitude+','+longitude+','+price+','+price_per+','+Housing_pattern_bedroom \
            +','+Housing_pattern_hall+','+Housing_pattern_toilet+','+age+','+floor+','+building_height \
            +','+building_type+','+elevator+','+orientation+','+area_total+','+main_building \
            +','+establishes_with_whole+','+parking_space+','+community+','+building_structure)
        ff.write('\n')
        f.close()
    except:
        print pk
ff.close()

In [58]:
#計算URL
line_num = 0
f = open("housefunex_detail.txt","r")
for line in f.readlines():
    line_num = line_num + 1
print line_num
f.close()

In [69]:
#將重複的物件讀出
case_dic = {}
f = open('housefun_URL.txt','r')
re = open('re_3.txt','w')
for line in f.readlines():
    rec = line.strip()
    caseno = rec.split('/')[5]
    if caseno not in case_dic:
        case_dic[caseno] = 1
    else:
        re.write(caseno+'\n')
re.close()
f.close()

In [7]:
#檢查資料是否重複(並產生新檔)

file_name = ['housefunXinbei_URL'] #原始檔案
file_path = '/Users/Derek/Project@iii/{}.txt' #檔案路徑,使用{}代表將數個檔案包覆


for File in file_name:  
    dic = []
    num = 0
    for line in open(file_path.format(File), 'r').readlines():
        #計算資料筆數
        num = num + 1
        
        if line not in dic:
            dic.append(line)
    file = open(file_path.format(File + '1'), 'w')
    for url in dic:
        file.write(url)
    file.close()
    
    print File
    print '原始資料筆數 : ' , num
    print '不重覆資料筆數 : ' , len(dic)
    print '重覆資料筆數 : ' , num - len(dic)

housefunXinbei_URL
原始資料筆數 :  16409
不重覆資料筆數 :  14478
重覆資料筆數 :  1931
